# SUV Analyse Schweiz

In [114]:
import pandas as pd
import numpy as np
import os
import re
import warnings
warnings.filterwarnings('ignore')

## Daten importieren

### Import Mockup Data
Daten sind auf 10000 zufällige Einträge reduziert. Aus Datenschutzgründen sind zusätzlich die die folgenden Spalten mit Platzhaltern Versehen:
- Erstinvekehrsetzung_Kanton
- Inverkehrsetzung_Kanton
- PLZ
- Ort

Bei der Spalte BFS-Gemeinde-Nr ist eine zufällige (valide) BFS Nummer angegeben.

In [115]:
## read astra data
df_input = pd.read_csv("Mockup_Data.csv", encoding = 'utf-8')

# reduce to Personenwagen
df = df_input.loc[df_input.Fahrzeugart == "Personenwagen"]

#a = df.sample(1000)
#a = df_res_raw.sample(1000)
#aa = df_res_raw.sort_index().head(1000)
#aaa = df.head(4000)
#sum(df_res_raw.index.duplicated(keep = False))
#aaaa = df_res_raw.loc[df_res_raw.index.duplicated(keep = False), :].sort_index()


# make all strings lowercasye
df.loc[:, 'Marke'] = df.loc[:, 'Marke'].astype(str).str.lower()
df.loc[:, 'Marke_und_Typ'] = df.loc[:, 'Marke_und_Typ'].astype(str).str.lower()
df.loc[:, 'Typ1'] = df.loc[:, 'Typ1'].astype(str).str.lower()
df.loc[:, 'Typ2'] = df.loc[:, 'Typ2'].astype(str).str.lower()
df.loc[:, 'Typ3'] = df.loc[:, 'Typ3'].astype(str).str.lower()

df.head(5)

,Unnamed: 0,Fahrzeugklasse,Fahrzeugart_Code,Fahrzeugart,Marken_Code,Marke,Marke_und_Typ,Typ1,Typ2,Typ3,...,PLZ,Ort,BFS-Gemeinde-Nr,Staat_Code,Staat,Halterart_Code,Halterart,Besondere_Verwendung_Code,Besondere_Verwendung,Unnamed: 63
0,4717617,M1,1,Personenwagen,6306,skoda,skoda yeti 1.2tsi,yeti 1.2tsi,yeti,1.2tsi,...,PLZ,Gemeinde,6235,CH,Schweiz,1,männlich,NaN,NaN,NaN
1,5424362,M1,1,Personenwagen,5100,toyota,toyota corolla 1.6,corolla 1.6,corolla,1.6,...,PLZ,Gemeinde,141,CH,Schweiz,1,männlich,NaN,NaN,NaN
2,6269855,M1,1,Personenwagen,4017,vw,vw sharan 2.0d 7,sharan 2.0d 7,sharan,2.0d,...,PLZ,Gemeinde,198,CH,Schweiz,3,juristisch,NaN,NaN,NaN
3,1365562,M1,1,Personenwagen,3019,fiat,fiat punto 1.4 8v,punto 1.4 8v,punto,1.4,...,PLZ,Gemeinde,2773,CH,Schweiz,2,weiblich,NaN,NaN,NaN
4,198970,M1,1,Personenwagen,4009,audi,audi a4 av. qu. 2.0t,a4 av. qu. 2.0t,a4,av.,...,PLZ,Gemeinde,3255,CH,Schweiz,2,weiblich,NaN,NaN,NaN


### Typenliste wird eingelesen

In [78]:
## read Typenliste
df_typen_input = pd.read_csv("Data_input/Typenliste SUV - export.csv", encoding = 'utf-8')
df_typen_input.columns
df_typen = df_typen_input.loc[df_typen_input.entfernen == "Nein"]

# make all strings lovercase
df_typen.loc[:, 'Marke'] = df_typen.loc[:, 'Marke'].astype(str).str.lower()
df_typen.loc[:, 'Typ'] = df_typen.loc[:, 'Typ'].astype(str).str.lower()

# define different searchtypes dfs
df_typen_alle = df_typen.loc[df_typen.such_typ == "alle"]
df_typen_normal = df_typen.loc[df_typen.such_typ == "normal"]
df_typen_exakt = df_typen.loc[df_typen.such_typ == "exakt"]

df_typen.head(5)

,Marke,Typ,entfernen,such_typ,original_typ,Lux
0,hummer,-,Nein,alle,Alle Modelle,lux
3,alfa romeo,stelvio,Nein,normal,STELVIO,lux
4,audi,q2,Nein,normal,Q2,x
5,bmw,x1,Nein,normal,X1,x
6,bmw,x4,Nein,normal,X4,lux


### Bevölkerungsdaten einlesen

In [79]:
## read Population data
df_pop = pd.read_csv("Data_input/Bevoelkerung_2018_export.csv", encoding = 'utf-8')

df_pop.head(5)

,BFS-Gemeinde-Nr,Ort_BfS,Bevölkerung
0,1,Aeugst am Albis,1982
1,2,Affoltern am Albis,12229
2,3,Bonstetten,5548
3,4,Hausen am Albis,3701
4,5,Hedingen,3734


## Typen zuordnen
Aus allen Personenwagen werden die SUVs gemäss der vorgegebenen Liste herausgesucht.

In [82]:
## all Hummers
df_res_all = pd.DataFrame()
for i, row in df_typen_alle.iterrows():
    print(row.Marke)
    marke = row.Marke
    index = [marke in x for x in list(df.Marke)]
    if(sum(index) != 0):
        res_temp = df.loc[index]
        res_temp.loc[:, 'Marke_harm'] = marke
        res_temp.loc[:, 'Type_harm'] = row.original_typ
        res_temp.loc[:, 'Luxury'] = row.Lux
        df_res_all = pd.concat([df_res_all, res_temp])

# normal cases (marke + typ)
df_res_normal = pd.DataFrame()
for marke in df_typen_normal.Marke.unique():
    temp_marke = df[[marke in x for x in list(df.Marke)]]
    df_typen_normal_marken = df_typen_normal[df_typen_normal.Marke == marke]
    print(marke + ": " + str(len(temp_marke)))

    for i, row in df_typen_normal_marken.iterrows():
        typ = row.Typ
        index = [typ in x for x in list(temp_marke.Marke_und_Typ)]
        if(sum(index) != 0):
            res_temp = temp_marke.loc[index]
            res_temp.loc[:, 'Marke_harm'] = row.Marke
            res_temp.loc[:, 'Type_harm'] = row.original_typ
            res_temp.loc[:, 'Luxury'] = row.Lux
            df_res_normal = pd.concat([df_res_normal, res_temp])
            print("   " + row.Typ + ": " + str(len(res_temp)))
            
## exakt cases (marke + typ)

df_res_exakt = pd.DataFrame()
for marke in df_typen_exakt.Marke.unique():
    temp_marke = df[[marke in x for x in list(df.Marke)]]
    df_typen_exakt_marken = df_typen_exakt[df_typen_exakt.Marke == marke]
    print(marke + ": " + str(len(temp_marke)))
        
    for i, row in df_typen_exakt_marken.iterrows():
        typ = row.Typ
        index_is = [(re.search(r"\b%s{1}\b" % typ, x) is not None) for x in list(temp_marke.Marke_und_Typ)]
        index_not1 = [(re.search(r"\b%s{1}\b" % "gt", x) is not None) for x in list(temp_marke.Marke_und_Typ)]
         # GT R nicht berücksichtigen!
        index = index_is & np.invert(index_not1)

        if(sum(index) != 0):
            res_temp = temp_marke.loc[index]
            res_temp.loc[:, 'Marke_harm'] = marke
            res_temp.loc[:, 'Type_harm'] = df_typen_exakt_marken.loc[i, 'original_typ']
            res_temp.loc[:, 'Luxury'] = row.Lux
            df_res_exakt = pd.concat([df_res_exakt, res_temp])
            print("   " + typ + ": " + str(len(res_temp)))
        
        
## concat
df_res_raw = pd.concat([df_res_all, df_res_normal, df_res_exakt])


hummer
alfa romeo: 73
   stelvio: 3
audi: 477
   q2: 10
   q3: 24
   q5: 49
   q7: 13
bmw: 461
   x1: 37
   x4: 7
   x3: 57
   x5: 29
   x6: 6
chevrolet: 48
   captiva: 5
   trax: 2
citroen: 225
   crosser: 5
   cactus: 6
dacia: 76
   duster: 17
daihatsu: 27
   terios: 12
   feroza: 1
fiat: 220
   sedici: 4
ford: 371
   ecosport: 6
   edge: 3
   kuga: 60
   maverick: 2
honda: 150
   cr-v: 30
   hr-v: 3
hyundai: 162
   35: 20
   55: 1
   kona: 4
   tucson: 13
   santa: 17
   terracan: 1
infiniti: 4
kia: 64
   niro: 1
   sportage: 11
   sorento: 8
lada: 0
lexus: 18
   rx: 4
mazda: 165
   cx-3: 8
   cx-5: 19
   cx-7: 3
mitsubishi: 97
   asx: 13
   outlander: 14
   pajero: 11
nissan: 164
   murano: 1
   qashqai: 55
   pathfinder: 5
   patrol: 2
   trail: 9
opel: 381
   antara: 7
   crossland: 5
   grandland: 5
   mokka: 18
   frontera: 1
peugeot: 275
   2008: 13
   4007: 2
   4008: 4
renault: 333
   captur: 19
   kadjar: 8
   koleos: 10
seat: 205
   ateca: 15
skoda: 328
   karoq: 5
   yeti

### Duplikate entfernen
Es gib einige wenige Duplikate, insbesondere der Hyundai Tucson / ix35 der teilweise als Tucson und ix35 bezeichnet wird und deshalb doppelt gefunden wird

In [111]:
# remove duplicates
    # df_duplicates = df_res_raw[df_res_raw.index.duplicated(keep = False)].sort_index()
df_res = df_res_raw[~df_res_raw.index.duplicated(keep = "first")]
df_res.loc[:, 'Marke_Typ_harm'] = df_res.Marke_harm + " - " + df_res.Type_harm

df_res.head(10)

,Unnamed: 0,Fahrzeugklasse,Fahrzeugart_Code,Fahrzeugart,Marken_Code,Marke,Marke_und_Typ,Typ1,Typ2,Typ3,...,Staat,Halterart_Code,Halterart,Besondere_Verwendung_Code,Besondere_Verwendung,Unnamed: 63,Marke_harm,Type_harm,Luxury,Marke_Typ_harm
792,2153606,M1,1,Personenwagen,5719,hummer,hummer h3 3.7 awd,h3 3.7 awd,h3,3.7,...,Schweiz,1,männlich,NaN,NaN,NaN,hummer,Alle Modelle,lux,hummer - Alle Modelle
258,76284,M1,1,Personenwagen,3003,alfa romeo,alfa romeo stelvio2.2jtdq4,stelvio2.2jtdq4,stelvio2.2jtdq4,nan,...,Schweiz,3,juristisch,NaN,NaN,NaN,alfa romeo,STELVIO,lux,alfa romeo - STELVIO
4106,75856,M1,1,Personenwagen,3003,alfa romeo,alfa romeo stelvio 2.0 q4,stelvio 2.0 q4,stelvio,2.0,...,Schweiz,1,männlich,NaN,NaN,NaN,alfa romeo,STELVIO,lux,alfa romeo - STELVIO
8945,77172,M1,1,Personenwagen,3003,alfa romeo,alfa-romeo stelvio 2.9 v6 q4,stelvio 2.9 v6 q4,stelvio,2.9,...,Schweiz,1,männlich,NaN,NaN,NaN,alfa romeo,STELVIO,lux,alfa romeo - STELVIO
2888,412855,M1,1,Personenwagen,4009,audi,audi q2 1.4tfsi,q2 1.4tfsi,q2,1.4tfsi,...,Schweiz,1,männlich,NaN,NaN,NaN,audi,Q2,x,audi - Q2
5330,412591,M1,1,Personenwagen,4009,audi,audi q2 2.0tfsi quatt.,q2 2.0tfsi quatt.,q2,2.0tfsi,...,Schweiz,1,männlich,NaN,NaN,NaN,audi,Q2,x,audi - Q2
6642,413689,M1,1,Personenwagen,4009,audi,audi q2 1.4tfsi,q2 1.4tfsi,q2,1.4tfsi,...,Schweiz,2,weiblich,NaN,NaN,NaN,audi,Q2,x,audi - Q2
7460,413805,M1,1,Personenwagen,4009,audi,audi q2 1.4tfsi,q2 1.4tfsi,q2,1.4tfsi,...,Schweiz,1,männlich,NaN,NaN,NaN,audi,Q2,x,audi - Q2
8683,412680,M1,1,Personenwagen,4009,audi,audi q2 30 tfsi,q2 30 tfsi,q2,30,...,Schweiz,2,weiblich,NaN,NaN,NaN,audi,Q2,x,audi - Q2
9128,414232,M1,1,Personenwagen,4009,audi,audi q2 1.6tdi,q2 1.6tdi,q2,1.6tdi,...,Schweiz,3,juristisch,NaN,NaN,NaN,audi,Q2,x,audi - Q2


In [102]:
# store results
df_res.to_csv("Data/output/res_mockup_29_11_2019.csv")

## Analyse

### SUVs pro Gemeinde zählen

In [103]:
## Group by Municipality

df_res_CH = df_res.loc[df_res.Staat_Code == "CH"]

counts_SUV = pd.DataFrame(df_res_CH.loc[:, 'BFS-Gemeinde-Nr'].value_counts())
counts_SUV.rename(columns={'BFS-Gemeinde-Nr':'Anzahl_SUV'}, inplace=True)
counts_SUV.loc[:, 'BFS-Gemeinde-Nr'] = counts_SUV.index

counts_SUV.head(5)

,Anzahl_SUV,BFS-Gemeinde-Nr
261,25,261
3203,19,3203
351,16,351
1061,14,1061
230,13,230


### Luxuriöse SUVs pro Gemeinde zählen

In [104]:

# add number of Luxury SUV
df_res_lux = df_res.loc[df_res.Luxury == "lux", :]
counts_luxSUV = pd.DataFrame(df_res_lux.loc[:, 'BFS-Gemeinde-Nr'].value_counts())
counts_luxSUV.rename(columns={'BFS-Gemeinde-Nr':'Anzahl_luxSUV'}, inplace=True)
counts_luxSUV.loc[:, 'BFS-Gemeinde-Nr'] = counts_luxSUV.index

counts_luxSUV.head(5)

,Anzahl_luxSUV,BFS-Gemeinde-Nr
261,10,261
3203,8,3203
6711,5,6711
230,5,230
3427,4,3427


### Gesamtanzahl Autos pro Gemeinde zählen

In [105]:
# add number of total cars
df_CH = df.loc[df.Staat_Code == "CH"]
counts_alleautos = pd.DataFrame(df_CH.loc[:, 'BFS-Gemeinde-Nr'].value_counts())
counts_alleautos.rename(columns={'BFS-Gemeinde-Nr':'Anzahl_auto'}, inplace=True)
counts_alleautos.loc[:, 'BFS-Gemeinde-Nr'] = counts_alleautos.index


counts_alleautos.head(5)

,Anzahl_auto,BFS-Gemeinde-Nr
261,197,261
6621,108,6621
351,92,351
5586,74,5586
230,66,230


### Zusammenführen mit Bevölkerungsdaten

In [106]:
# Join population
res_gemeinde = pd.merge(counts_SUV, df_pop, how = 'left')
res_gemeinde.loc[:, 'SUV_per_pop'] = res_gemeinde.Anzahl_SUV / res_gemeinde.Bevölkerung

# Join total number of cars
res_gemeinde = pd.merge(res_gemeinde, counts_alleautos, how = 'left')
res_gemeinde.loc[:, 'SUV_per_auto'] = res_gemeinde.Anzahl_SUV / res_gemeinde.Anzahl_auto

# Join number luxury SUV
res_gemeinde = pd.merge(res_gemeinde, counts_luxSUV, how = 'left')
res_gemeinde.loc[:, 'luxSUV_per_auto'] = res_gemeinde.Anzahl_luxSUV / res_gemeinde.Anzahl_auto
res_gemeinde.loc[:, 'luxSUV_per_pop'] = res_gemeinde.Anzahl_luxSUV / res_gemeinde.Bevölkerung

# remove Liechtenstein
res_gemeinde = res_gemeinde.loc[res_gemeinde.loc[:, 'BFS-Gemeinde-Nr'] < 7000, :]


## Resultate

Erinnerung: Diese Daten sind zufällig gemischt und zeigen nur einen kleinen Ausschnitt aus allen Daten (10'000 Einträge von ca. 4.5 Millionen)

### Tabelle mit allen Gemeinden 



In [107]:
# store results
res_gemeinde.to_csv("Data_output/karte_gemeinden_mockup_28112019.csv", index = False)
res_gemeinde.head(5)

,Anzahl_SUV,BFS-Gemeinde-Nr,Ort_BfS,Bevölkerung,SUV_per_pop,Anzahl_auto,SUV_per_auto,Anzahl_luxSUV,luxSUV_per_auto,luxSUV_per_pop
0,25,261,Zürich,415367.0,0.000060,197,0.126904,10.0,0.050761,0.000024
1,19,3203,St. Gallen,75833.0,0.000251,55,0.345455,8.0,0.145455,0.000105
2,16,351,Bern,133883.0,0.000120,92,0.173913,3.0,0.032609,0.000022
3,14,1061,Luzern,81691.0,0.000171,59,0.237288,2.0,0.033898,0.000024
4,13,230,Winterthur,111851.0,0.000116,66,0.196970,5.0,0.075758,0.000045


### Tabelle mit beliebtesten SUV Marken

In [112]:
counts_marke = df_res.Marke_harm.value_counts().sort_values(ascending = False)
counts_marke.to_csv("Data_output/Top_Marken_Mockup_28112019.csv", index = True)
counts_marke.head(5)

bmw       136
vw        117
audi       96
nissan     72
ford       71
Name: Marke_harm, dtype: int64

### Tabelle mit beliebtesten SUV Typen

In [113]:
counts_marke_type = df_res.Marke_Typ_harm.value_counts().sort_values(ascending = False)
counts_marke_type.to_csv("Data_output/Top_Typen_Mockup_28112019.csv", index = True)
counts_marke_type.head(5)

vw - TIGUAN         82
ford - KUGA         60
bmw - X3            57
nissan - QASHQAI    55
audi - Q5           49
Name: Marke_Typ_harm, dtype: int64